## Better Way 43
### 커스텀 컨테이너 타입은 collections.abc를 상속하라

### [collections.abc](https://docs.python.org/ko/3/library/collections.abc.html)
* 컨테이너 타입에 정의해야 하는 전형적인 메서드를 모두 제공하는 추상 베이스 클래스(abstract base class)들의 모음 
* 하위 클래스에 필요한 메서드의 구현이 안 되어 있으면 에러로 알려준다.

In [8]:
### 멤버들의 빈도를 계산하는 메서드가 포함된 커스텀 리스트 타입

# FrequencyList가 List의 하위 클래스이므로 List가 제공하는 모든 표준함수를 사용할 수 있다.
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    
    # 필요한 기능을 제공하는 메서드를 얼마든지 추가 가능     
    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

In [9]:
foo = FrequencyList(['a', 'b', 'a', 'c','b','a','d'])
print('길이: ', len(foo))

foo.pop()
print('pop한 다음: ', repr(foo))
print('빈도:', foo.frequency())

길이:  7
pop한 다음:  ['a', 'b', 'a', 'c', 'b', 'a']
빈도: {'a': 3, 'b': 2, 'c': 1}


 ### 아래의 클래스가 시퀀스 타입처럼 작동하게 하는 방법은?
 : 트리 노드를 깊이 '우선 순회(depth first traverse)'하는 커스텀 \__getitem\__ 메서드 구현을 제공한다.

In [10]:
# 리스트 같고 인덱싱가능하지만, 리스트의 하위클래스는 아닌 클래스

# 이진트리 클래스
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

#### 인덱스 기능을 구현할 때 \__getitem\__ 사용하듯

In [11]:
# 인덱스를 사용해 시퀀스에 접근하는 코드
bar = [1, 2, 3]
print(bar[0])

#아래의 특별 메서드로 해석된다.
print(bar.__getitem__(0))

1
1


#### 이진트리를 순회하는 것도 \__getitem\__메서드를 구현하자

In [12]:
# 커스텀 __getitme__메서드 구현
class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()
            
    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i == index:
                return item.value
        raise IndexError(f'인덱스 범위 초과: {index}')

In [13]:
# 이진트리
tree = IndexableNode(
    10,
    left = IndexableNode(
        5, 
        left = IndexableNode(2),
        right = IndexableNode(
        6,
        right = IndexableNode(7))),
    right = IndexableNode(
        15,
        left = IndexableNode(11)))

In [14]:
print('LRR: ', tree.left.right.right.value)
print('인덱스 0:', tree[0])
print('인덱스 1:', tree[1])
print('11이 트리 안에 있나?', 11 in tree)
print('17이 트리 안에 있나?', 17 in tree)
print('트리: ', list(tree))

LRR:  7
인덱스 0: 2
인덱스 1: 5
11이 트리 안에 있나? True
17이 트리 안에 있나? False
트리:  [2, 5, 6, 7, 10, 11, 15]


### 문제: __getitem__을 구현하는 것 만으로는 리스트 인스턴스에서 기대할 수 있는 모든 시퀀스 의미구조를 제공할 수 없다.

In [15]:
# len 내장함수는 __len__ 특별메서드를 구현해야 제대로 작동.
len(tree)

TypeError: object of type 'IndexableNode' has no len()

In [16]:
# 이진트리의 길이를 반환하는 메서드를 추가한 IndexableNode 하위 클래스
class SequenceNode(IndexableNode):
    # 커스텀 시퀀스타입은 꼭 이 메서드를 구현해야 한다.
    def __len__(self):   
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count
    
tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(
            6,
            right=SequenceNode(7))),
    right=SequenceNode(
        15,
        left=SequenceNode(11))
)

print('트리 길이:', len(tree))

트리 길이: 7


### 그러나 \__getitem\__, \__len\__만으로는 이 클래스가 올바른 시퀀스라 할수 없다.

#### count, index 등등 더많은 메서드들 필요

<br>

### 그럼 collections.abc 모듈을 사용하자!

In [17]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

# 필요한 메서드가 구현되지 않으면 아래의 에러가 발생한다.
foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

#### SequenceNode에서 한 것처럼,    
#### collections.abc에서 가져온 추상 기반 클래스가 요구하는 모든 메서드를 구현하면...

#### index, count등의 추가 메서드 구현을 거저 얻는다.

In [23]:
class BetterNode(SequenceNode, Sequence): # Sequence를 상속받지 않으면 index 메소드가 없다는 에러 발생
    pass

tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(
            6,
            right=BetterNode(7))),
     right=BetterNode(
         5,
         left=BetterNode(11))
)

print('7의 인덱스:', tree.index(7))
print('10의 개수:', tree.count(10))

7의 인덱스: 3
10의 개수: 1


### 추상 기반 클래스의 이점은 필수적인 특별메서드가 더 많은 복잡한 컨테이너타입을 구현할 때 더 커진다.
#### ex) Set, MutableMapping 

### collection.abc 모듈 외에도 특별메서드는 여러 종류가 있다.
#### 컨테이너/비컨테이너 모두 특별메서드 구현 가능하다.

## 요약
* **간편하게 쓸 때는 파이썬 컨테이너 타입(내장)**을 직접 상속하라.
* 커스텀 컨테이너를 제대로 구현하려면 갖춰야 할 메서드가 매우 많다.
* **커스텀 컨테이너의 타입이 collection.abc에 정의된 인터페이스를 상속**하면,
  정상 작동에 **필요한 인터페이스와 기능을 제대로 구현하도록 보장**할 수 있다.